In [2]:
#!unzip /content/train.zip

Archive:  /content/train.zip
   creating: modified_brown/
  inflating: modified_brown/ca01     
  inflating: modified_brown/ca02     
  inflating: modified_brown/ca03     
  inflating: modified_brown/ca04     
  inflating: modified_brown/ca05     
  inflating: modified_brown/ca06     
  inflating: modified_brown/ca07     
  inflating: modified_brown/ca08     
  inflating: modified_brown/ca09     
  inflating: modified_brown/ca10     
  inflating: modified_brown/ca11     
  inflating: modified_brown/ca12     
  inflating: modified_brown/ca13     
  inflating: modified_brown/ca14     
  inflating: modified_brown/ca15     
  inflating: modified_brown/ca16     
  inflating: modified_brown/ca17     
  inflating: modified_brown/ca18     
  inflating: modified_brown/ca19     
  inflating: modified_brown/ca20     
  inflating: modified_brown/ca21     
  inflating: modified_brown/ca22     
  inflating: modified_brown/ca23     
  inflating: modified_brown/ca24     
  inflating: modified_brown/ca

In [3]:
import os
import sys
import numpy as np

In [13]:
def load_corpus(path):
    sentences = []
    count = 0
    for sample in os.listdir(path):
      with open(path+"/" +sample, "r", errors="ignore") as f:
        data = f.read()
      for line in data:
        sentence = data.strip().split(" ")
        tagged_sentence = []
        for word in sentence:
          tagged_sentence.append(tuple(word.split("/")))
      sentences.append(tagged_sentence)
    return sentences

#corpus = load_corpus("/content/modified_brown")
folder_name = input("Input path: ")
sentences = load_corpus(folder_name)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [14]:
tags = {"ADJECTIVE": 1, "NOUN": 2, "PUNCT":3, "PRONOUN":4, "VERB":5, "PREPOSITION":6, "DETERMINER":7, "CONJUNCTION":8, "ADVERB":9, "NUMBER":10, "X":11}
tokens = {}

def create_dataset(sentences):
  count = 0
  for sentence in sentences:
    for entry in sentence:
      token = entry[0].lower()
      if token not in tokens:
        count += 1
        tokens[token] = count
  count += 1
  tokens["[PAD]"] = count
  train_X = []
  train_Y = []
  for sentence in sentences:
    train_X_sentence = []
    train_Y_sentence = []
    for entry in sentence:
      train_X_sentence.append(tokens[entry[0].lower()])
      train_Y_sentence.append(tags[entry[1]])
    train_X.append(train_X_sentence)
    train_Y.append(train_Y_sentence)
  train_X = np.array(train_X)
  train_Y = np.array(train_Y)
  return train_X, train_Y 

train_X, train_Y = create_dataset(sentences)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [15]:
import tensorflow as tf
from tensorflow import keras

In [34]:
from keras_preprocessing.sequence import pad_sequences as pad

def pad_sequences(train_X, train_Y):
  MAX_LENGTH = len(train_X[0])
  train_X = pad(train_X, padding = 'post', maxlen = MAX_LENGTH)
  train_Y = pad(train_Y, padding = 'post', maxlen = MAX_LENGTH, value = 11)  
  return train_X, train_Y, MAX_LENGTH

train_X, train_Y, MAX_LENGTH = pad_sequences(train_X, train_Y)
print(train_Y[0:10])

[[ 7  1  3 ...  4  5  3]
 [ 2  4  2 ...  9  1  3]
 [ 6  4  3 ...  0  0  0]
 ...
 [11  5 10 ...  0  0  0]
 [ 7  2  6 ...  0  0  0]
 [ 4  2  6 ...  7  2  3]]


In [42]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam

def define_model(MAX_LENGTH):
  model = Sequential()
  model.add(InputLayer(input_shape=(MAX_LENGTH, )))
  model.add(Embedding(len(tokens), 128))
  model.add(Bidirectional(LSTM(256, return_sequences=True)))
  model.add(TimeDistributed(Dense(12)))
  model.add(Activation('softmax'))
  
  model.compile(loss='categorical_crossentropy',
                optimizer=Adam(0.001),
                metrics=['accuracy'])
  
  model.summary()
  print(model.summary())
  return model

model = define_model(MAX_LENGTH)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 2403, 128)         7268096   
                                                                 
 bidirectional_5 (Bidirectio  (None, 2403, 512)        788480    
 nal)                                                            
                                                                 
 time_distributed_5 (TimeDis  (None, 2403, 12)         6156      
 tributed)                                                       
                                                                 
 activation_5 (Activation)   (None, 2403, 12)          0         
                                                                 
Total params: 8,062,732
Trainable params: 8,062,732
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_5"
______________________

In [37]:
from keras.utils import to_categorical
print(train_Y[0:10])
encoded_train_Y = to_categorical(train_Y, num_classes = 12)

[[ 7  1  3 ...  4  5  3]
 [ 2  4  2 ...  9  1  3]
 [ 6  4  3 ...  0  0  0]
 ...
 [11  5 10 ...  0  0  0]
 [ 7  2  6 ...  0  0  0]
 [ 4  2  6 ...  7  2  3]]


In [43]:
def train(model, train_X, train_Y):
  model.fit(x=train_X, y=train_Y, batch_size = 128, epochs = 40, validation_split = 0.2)
  return model

model = train(model, train_X, encoded_train_Y)

Epoch 1/40
4/4 [==============================] - 12s 852ms/step - loss: 2.4685 - accuracy: 0.2168 - val_loss: 2.4022 - val_accuracy: 0.2624
Epoch 2/40
4/4 [==============================] - 2s 606ms/step - loss: 2.3346 - accuracy: 0.2637 - val_loss: 2.1166 - val_accuracy: 0.2582
Epoch 3/40
4/4 [==============================] - 3s 615ms/step - loss: 2.1395 - accuracy: 0.2344 - val_loss: 2.1245 - val_accuracy: 0.1895
Epoch 4/40
4/4 [==============================] - 3s 632ms/step - loss: 2.1125 - accuracy: 0.1882 - val_loss: 2.0495 - val_accuracy: 0.3326
Epoch 5/40
4/4 [==============================] - 3s 656ms/step - loss: 2.0418 - accuracy: 0.2893 - val_loss: 2.0361 - val_accuracy: 0.2625
Epoch 6/40
4/4 [==============================] - 3s 621ms/step - loss: 2.0303 - accuracy: 0.2640 - val_loss: 2.0013 - val_accuracy: 0.2626
Epoch 7/40
4/4 [==============================] - 3s 619ms/step - loss: 2.0000 - accuracy: 0.2648 - val_loss: 1.9762 - val_accuracy: 0.3146
Epoch 8/40
4/4 [===

In [86]:
def test(model, sentence):
  if type(sentence) == type("string"):
    sentence = sentence.strip().split(" ")
  tags = {1:"ADJECTIVE", 2:"NOUN", 3:"PUNCT", 4:"PRONOUN", 5:"VERB", 6:"PREPOSITION", 7:"DETERMINER", 8:"CONJUNCTION", 9:"ADVERB", 10:"NUMBER", 11:"OTHER", 0:"PADDING"}
  tokens = {}
  count = 0
  for token in sentence:
    if token not in tokens:
      count += 1
      tokens[token] = count
  test_X = []
  for token in sentence:
    test_X.append(tokens[token])
  test_X = pad([test_X], maxlen=MAX_LENGTH, padding='post')
  prediction = model.predict(test_X)
  predicted_tags = []
  for predicted_value in prediction[0]:
    predicted_tags.append(tags[np.argmax(predicted_value[1:])+1])
  return predicted_tags[:len(sentence)]

testString1 = ["the", "secretariat", "is", "expected", "to", "race", "tomorrow", "."]
print(test(model, testString1))

testString2 = "people continue to enquire the reason for the race for outer space ."
print(test(model, testString2))

1/1 [==============================] - 0s 189ms/step
['DETERMINER', 'DETERMINER', 'PUNCT', 'ADJECTIVE', 'NOUN', 'VERB', 'ADJECTIVE', 'CONJUNCTION']
1/1 [==============================] - 0s 194ms/step
['DETERMINER', 'ADJECTIVE', 'PUNCT', 'ADJECTIVE', 'NOUN', 'VERB', 'ADJECTIVE', 'NOUN', 'CONJUNCTION', 'ADJECTIVE', 'PRONOUN', 'VERB', 'PRONOUN']
